In [1]:
import logging
import os
import pathlib
import sys
import time
import urllib

import numpy as np
import pandas as pd
import psutil
from arg_parsing_utils import check_for_missing_args, parse_args
from loading_classes import ImageSetLoader, ObjectLoader
from notebook_init_utils import bandicoot_check, init_notebook
from resource_profiling_util import get_mem_and_time_profiling
from sammed3d_featurizer import call_SAMMed3D_pipeline

root_dir, in_notebook = init_notebook()

In [2]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [3]:
if not in_notebook:
    arguments_dict = parse_args()
    patient = arguments_dict["patient"]
    well_fov = arguments_dict["well_fov"]
    compartment = arguments_dict["compartment"]
    channel = arguments_dict["channel"]
    input_subparent_name = arguments_dict["input_subparent_name"]
    mask_subparent_name = arguments_dict["mask_subparent_name"]
    output_features_subparent_name = arguments_dict["output_features_subparent_name"]

else:
    well_fov = "C4-2"
    patient = "NF0014_T1"
    compartment = "Nuclei"
    channel = "DNA"
    input_subparent_name = "zstack_images"
    mask_subparent_name = "segmentation_masks"
    output_features_subparent_name = "extracted_features"

image_set_path = pathlib.Path(
    f"{root_dir}/data/{patient}/{input_subparent_name}/{well_fov}/"
)
mask_set_path = pathlib.Path(
    f"{root_dir}/data/{patient}/{mask_subparent_name}/{well_fov}/"
)

output_parent_path = pathlib.Path(
    f"{root_dir}/data/{patient}/{output_features_subparent_name}/{well_fov}/"
)
output_parent_path.mkdir(parents=True, exist_ok=True)

In [4]:
sam3dmed_checkpoint_url = (
    "https://huggingface.co/blueyo0/SAM-Med3D/resolve/main/sam_med3d_turbo.pth"
)
sam3dmed_checkpoint_path = pathlib.Path("../models/sam-med3d-turbo.pth").resolve()
if not sam3dmed_checkpoint_path.exists():
    sam3dmed_checkpoint_path.parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(sam3dmed_checkpoint_url, str(sam3dmed_checkpoint_path))

In [5]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [6]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [7]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    mask_set_path=mask_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [8]:
# load the objects for the compartment and channel of interest
object_loader = ObjectLoader(
    image_set_loader.image_set_dict[channel],
    image_set_loader.image_set_dict[compartment],
    channel,
    compartment,
)
#  redirect stdout to logging
logging.info("Starting SAM-Med3D feature extraction")
feature_dict = call_SAMMed3D_pipeline(
    object_loader=object_loader,
    SAMMed3D_model_path=str(sam3dmed_checkpoint_path),
    feature_type="cls",
)

INFO:root:Starting SAM-Med3D feature extraction


creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model SAM-Med3D
try to load pretrained weights from /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/3.cellprofiling/models/sam-med3d-turbo.pth
creating model S

In [9]:
final_df = pd.DataFrame(feature_dict)

final_df["feature_name"] = (
    final_df["feature_name"] + "_" + final_df["compartment"] + "_" + final_df["channel"]
)
final_df["feature_name"] = final_df["feature_name"].str.replace("_feature_", ".")
final_df = final_df.drop(columns=["compartment", "channel"])
final_df.head()

,object_id,feature_name,value,feature_type
0,2,SAMMed3D.0_Nuclei_DNA,-0.223103,cls
1,2,SAMMed3D.1_Nuclei_DNA,-0.296578,cls
2,2,SAMMed3D.2_Nuclei_DNA,-0.035843,cls
3,2,SAMMed3D.3_Nuclei_DNA,-0.002480,cls
4,2,SAMMed3D.4_Nuclei_DNA,-0.101575,cls


In [10]:
final_df = final_df.pivot(
    index="object_id", columns="feature_name", values="value"
).reset_index()
# drop the multiindexing from pivot
final_df.columns.name = None
final_df.head()

,object_id,SAMMed3D.0_Nuclei_DNA,SAMMed3D.100_Nuclei_DNA,SAMMed3D.101_Nuclei_DNA,SAMMed3D.102_Nuclei_DNA,SAMMed3D.103_Nuclei_DNA,SAMMed3D.104_Nuclei_DNA,SAMMed3D.105_Nuclei_DNA,SAMMed3D.106_Nuclei_DNA,SAMMed3D.107_Nuclei_DNA,...,SAMMed3D.91_Nuclei_DNA,SAMMed3D.92_Nuclei_DNA,SAMMed3D.93_Nuclei_DNA,SAMMed3D.94_Nuclei_DNA,SAMMed3D.95_Nuclei_DNA,SAMMed3D.96_Nuclei_DNA,SAMMed3D.97_Nuclei_DNA,SAMMed3D.98_Nuclei_DNA,SAMMed3D.99_Nuclei_DNA,SAMMed3D.9_Nuclei_DNA
0,2,-0.223103,0.001824,-0.167292,0.179799,0.044472,-0.097601,-0.219815,-0.040644,-0.067726,...,-0.114832,0.015723,-0.010783,0.029430,0.050556,0.068318,0.189328,0.315357,0.201357,0.100042
1,18,-0.229251,-0.002924,-0.158431,0.180910,0.046183,-0.109067,-0.209736,-0.042766,-0.061397,...,-0.110946,0.030395,-0.010797,0.031675,0.041927,0.059795,0.182370,0.299851,0.205252,0.098277
2,28,-0.243821,-0.004795,-0.159623,0.154213,0.055733,-0.087381,-0.218884,-0.034762,-0.063357,...,-0.108660,0.046060,-0.010798,0.030660,0.053262,0.061074,0.201527,0.317599,0.216324,0.116264
3,29,-0.212849,0.000584,-0.158901,0.173757,0.062115,-0.099460,-0.203697,-0.029119,-0.053995,...,-0.121236,0.024075,-0.010806,0.029800,0.053676,0.066347,0.186779,0.320616,0.197048,0.101015
4,33,-0.233716,-0.002256,-0.172662,0.164734,0.051692,-0.096874,-0.212065,-0.035654,-0.059578,...,-0.097257,0.036958,-0.010835,0.029817,0.048002,0.059939,0.191464,0.313583,0.215913,0.111997


In [11]:
# prepend compartment and channel to column names
for col in final_df.columns:
    if col not in ["object_id"]:
        final_df[col] = final_df[col].astype(np.float32)

In [12]:
# de-fragment
final_df = final_df.copy()
final_df.insert(1, "image_set", image_set_loader.image_set_name)

output_file = pathlib.Path(
    output_parent_path / f"SAMMed3D_{compartment}_{channel}_GPU_features.parquet"
)
final_df.to_parquet(output_file, index=False)
final_df.head()

,object_id,image_set,SAMMed3D.0_Nuclei_DNA,SAMMed3D.100_Nuclei_DNA,SAMMed3D.101_Nuclei_DNA,SAMMed3D.102_Nuclei_DNA,SAMMed3D.103_Nuclei_DNA,SAMMed3D.104_Nuclei_DNA,SAMMed3D.105_Nuclei_DNA,SAMMed3D.106_Nuclei_DNA,...,SAMMed3D.91_Nuclei_DNA,SAMMed3D.92_Nuclei_DNA,SAMMed3D.93_Nuclei_DNA,SAMMed3D.94_Nuclei_DNA,SAMMed3D.95_Nuclei_DNA,SAMMed3D.96_Nuclei_DNA,SAMMed3D.97_Nuclei_DNA,SAMMed3D.98_Nuclei_DNA,SAMMed3D.99_Nuclei_DNA,SAMMed3D.9_Nuclei_DNA
0,2,C4-2,-0.223103,0.001824,-0.167292,0.179799,0.044472,-0.097601,-0.219815,-0.040644,...,-0.114832,0.015723,-0.010783,0.029430,0.050556,0.068318,0.189328,0.315357,0.201357,0.100042
1,18,C4-2,-0.229251,-0.002924,-0.158431,0.180910,0.046183,-0.109067,-0.209736,-0.042766,...,-0.110946,0.030395,-0.010797,0.031675,0.041927,0.059795,0.182370,0.299851,0.205252,0.098277
2,28,C4-2,-0.243821,-0.004795,-0.159623,0.154213,0.055733,-0.087381,-0.218884,-0.034762,...,-0.108660,0.046060,-0.010798,0.030660,0.053262,0.061074,0.201527,0.317599,0.216324,0.116264
3,29,C4-2,-0.212849,0.000584,-0.158901,0.173757,0.062115,-0.099460,-0.203697,-0.029119,...,-0.121236,0.024075,-0.010806,0.029800,0.053676,0.066347,0.186779,0.320616,0.197048,0.101015
4,33,C4-2,-0.233716,-0.002256,-0.172662,0.164734,0.051692,-0.096874,-0.212065,-0.035654,...,-0.097257,0.036958,-0.010835,0.029817,0.048002,0.059939,0.191464,0.313583,0.215913,0.111997


In [13]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    feature_type="SAMMed3D",
    well_fov=well_fov,
    patient_id=patient,
    channel="DNA",
    compartment=compartment,
    CPU_GPU="GPU",
    output_file_dir=pathlib.Path(
        f"{root_dir}/data/{patient}/extracted_features/run_stats/{well_fov}_SAMMed3D_{channel}_{compartment}_GPU.parquet"
    ),
)


        Memory and time profiling for the run:

        Patient ID: NF0014_T1

        Well and FOV: C4-2

        Feature type: SAMMed3D

        CPU/GPU: GPU")

        Memory usage: 2213.39 MB

        Time:

        --- %s seconds --- % 81.41367220878601

        --- %s minutes --- % 1.3568945368131002

        --- %s hours --- % 0.022614908946885004
    


True